In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from zipfile import ZipFile
import os

pd.options.display.float_format = '{:.0f}'.format # Supress scientific notation

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip -nv

with ZipFile('book-crossings.zip', 'r') as zObject:
    zObject.extractall()

os.remove('book-crossings.zip')

filenames = ['BX-Book-Ratings.csv', 'BX-Books.csv', 'BX-Users.csv']

2024-01-11 23:26:15 URL:https://cdn.freecodecamp.org/project-data/books/book-crossings.zip [26085508/26085508] -> "book-crossings.zip" [1]


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    filenames[1],
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    filenames[0],
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

for file in filenames:
    os.remove(file)

In [4]:
df = df_ratings.merge(right = df_books, how = 'left')

df.describe()

,user,rating
count,1149780,1149780
mean,140386,3
std,80562,4
min,2,0
25%,70345,0
50%,141010,0
75%,211028,7
max,278854,10


In [5]:
df.sample(10)

,user,isbn,rating,title,author
349440,83637,0061097357,0,The Floating Girl,Sujata Massey
31819,7346,0399139745,0,Wild Horses,Dick Francis
915044,223087,0385486693,0,Evil Breeding: A Dog Lover's Mystery,Susan Conant
458954,110076,1568842422,0,Mosaic for Dummies Windows Edition (For Dummies),David Angell
924805,225087,0373244770,0,The Royal Macallister (The Baby Bet: Macalli...,Joan Elliot Pickart
1137598,273971,0070542023,0,Basic Scientific Subroutines,Fred Ruckdeschel
212056,49177,0394717368,0,Rules for Radicals,Saul D. Alinsky
169768,36836,0843105690,0,Little Mouse on the Prairie (Serendipity Books...,Stephan Cosgrove
575017,138393,0930031113,10,Permanent New Yorkers: A Biographical Guide to...,Judi Culbertson
418187,99630,0060965924,5,The Official Cat I.Q. Test,Peter Mandel


In [6]:
# Get list of users that posted 200+ reviews
users = df.groupby('user').count().title.sort_values()
users_few = users.loc[users >= 200].index.to_list()

# Get list of books that have 100+ reviews
books = df.groupby('title').count().user.sort_values()
books_few = books.loc[books >= 100].index.to_list()

# Filter dataframe with previously generated lists
df = df.loc[(df.title.isin(books_few)) & (df.user.isin(users_few))]

df.describe()

,user,rating
count,66730,66730
mean,140319,2
std,81015,4
min,254,0
25%,69697,0
50%,139742,0
75%,212923,2
max,278418,10


In [7]:
df

,user,isbn,rating,title,author
1456,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
1468,277427,006092988X,0,A Tree Grows in Brooklyn,Betty Smith
1469,277427,0060930535,0,The Poisonwood Bible: A Novel,Barbara Kingsolver
1471,277427,0060934417,0,Bel Canto: A Novel,Ann Patchett
1474,277427,0061009059,9,One for the Money (Stephanie Plum Novels (Pape...,Janet Evanovich
...,...,...,...,...,...
1147441,275970,1400031362,0,Morality for Beautiful Girls (No.1 Ladies Dete...,Alexander McCall Smith
1147516,275970,1573227951,0,An Instance of the Fingerpost,Iain Pears
1147517,275970,1573229725,0,Fingersmith,Sarah Waters
1147556,275970,1586210661,9,Me Talk Pretty One Day,David Sedaris


In [8]:
df = df.drop_duplicates(['title','user'], keep='first')

In [9]:
# Convert dataframe into 2D matrix
matrix = df.pivot_table(index='title', columns='user', values='rating').fillna(0)

matrix.head()

user,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9,0,0,0,0,0,0,0,0,0,...,10,0,0,0,0,0,0,0,0,0
1st to Die: A Novel,0,0,0,0,0,0,0,0,0,9,...,0,0,0,0,0,0,0,0,0,0
24 Hours,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,10,0,0
2nd Chance,0,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4 Blondes,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
N_RECOMMENDS = 5

In [11]:
neighbors = NearestNeighbors(algorithm='brute', n_neighbors=N_RECOMMENDS + 1)
neighbors.fit(matrix)

NearestNeighbors(algorithm='brute', n_neighbors=6)

In [12]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  list = [book, []]
  
  if book in df['title'].values:
    distances, indices = neighbors.kneighbors(matrix.loc[[book]])
    
    for i in range(1, N_RECOMMENDS + 1):
      recomm = [matrix.index[indices.flatten()[i]], distances.flatten()[i]]
      list[1].append(recomm)
  
  return list

In [13]:
get_recommends('The Queen of the Damned (Vampire Chronicles (Paperback))')

['The Queen of the Damned (Vampire Chronicles (Paperback))',
 [['No Safe Place', 35.4682957019364],
  ['The Vendetta Defense', 35.70714214271425],
  ['Long After Midnight', 35.749125863438955],
  ['Jacob Have I Loved', 35.94440151122286],
  ['The Prometheus Deception', 36.193922141707716]]]

In [14]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Perks of Being a Wallflower', 60.76183012385325], ['Silent Night : A Christmas Suspense Story', 60.78651166171653], ['Blue Diary', 60.83584469702052], ['Gap Creek: The Story Of A Marriage', 60.860496218811754], ['Jacob Have I Loved', 60.88513775955508]]]
You haven't passed yet. Keep trying!
